# **Deliverable 2: Determine Bias of Vine Reviews**

Using your knowledge of PySpark, Pandas, or SQL, you’ll determine if there is any bias towards reviews that were written as part of the Vine program. For this analysis, you'll determine if having a paid Vine review makes a difference in the percentage of 5-star reviews.

Using either PySpark, Pandas, or SQL, follow the instructions below to complete Deliverable 2.



1.   Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.
2.   Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.

*   If you use the SQL option below, you’ll need to cast your columns as floats using WHERE CAST(helpful_votes AS FLOAT)/CAST(total_votes AS FLOAT) >=0.5.


3.  Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
4. Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
5. Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).



**NOTE**

We recommend using either PySpark or Pandas to perform the analysis. Using SQL queries is more challenging, especially for Step 5 above, and is better suited for intermediate and experienced SQL programmers.

# **Using PySpark**


1.   Create a new Google Colab Notebook, and name it Vine_Review_Analysis.
2.   Extract the dataset you used in Deliverable 1.
3. Recreate the vine_table, and perform your analysis using the steps above.
4. Export your Vine_Review_Analysis Google Colab Notebook as an ipynb file, and save it to your Amazon_Vine_Analysis GitHub repository.






In [2]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [91.1 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [912 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports In

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

# **Load Amazon Data into Spark dataframe (df--same as Amazazon_Reviews_ETL.ipynb)**

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Shoes_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...| 2015-08-31|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...|           Shoes|          5|    

In [24]:
ratings_df = df.select(["vine", "verified_purchase", "star_rating", "total_votes", "helpful_votes"])
ratings_df.show()

+----+-----------------+-----------+-----------+-------------+
|vine|verified_purchase|star_rating|total_votes|helpful_votes|
+----+-----------------+-----------+-----------+-------------+
|   N|                Y|          1|          0|            0|
|   N|                Y|          5|          0|            0|
|   N|                Y|          4|          0|            0|
|   N|                Y|          5|          6|            0|
|   N|                Y|          3|          0|            0|
|   N|                Y|          5|          1|            1|
|   N|                Y|          5|          1|            1|
|   N|                Y|          4|          0|            0|
|   N|                Y|          5|          0|            0|
|   N|                Y|          3|          0|            0|
|   N|                Y|          5|          0|            0|
|   N|                Y|          5|          0|            0|
|   N|                Y|          4|          1|       

In [25]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.
votes_df = ratings_df.filter(ratings_df["total_votes"] >= 20)

# Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50% (0.5).
filtered_votes_df = votes_df.filter(votes_df["helpful_votes"]/votes_df["total_votes"] >= 0.5)
filtered_votes_df.show()

+----+-----------------+-----------+-----------+-------------+
|vine|verified_purchase|star_rating|total_votes|helpful_votes|
+----+-----------------+-----------+-----------+-------------+
|   N|                Y|          5|         49|           45|
|   N|                Y|          5|         25|           25|
|   N|                N|          5|         27|           16|
|   N|                Y|          5|         21|           19|
|   N|                Y|          2|         22|           19|
|   N|                Y|          5|         30|           30|
|   N|                Y|          5|         29|           28|
|   N|                Y|          5|         28|           28|
|   N|                Y|          2|         51|           43|
|   N|                Y|          5|         22|           21|
|   N|                Y|          1|         33|           33|
|   N|                Y|          2|         76|           73|
|   N|                Y|          3|         26|       

In [26]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
from pyspark.sql.functions import col, avg
paid_df = filtered_votes_df.filter(filtered_votes_df['vine']== 'Y')
paid_df.show()

+----+-----------------+-----------+-----------+-------------+
|vine|verified_purchase|star_rating|total_votes|helpful_votes|
+----+-----------------+-----------+-----------+-------------+
|   Y|                N|          5|         22|           21|
|   Y|                N|          5|         38|           34|
|   Y|                N|          4|        184|          180|
|   Y|                N|          4|         21|           21|
|   Y|                N|          5|         98|           88|
|   Y|                N|          5|         26|           24|
|   Y|                N|          5|         28|           27|
|   Y|                N|          5|         56|           53|
|   Y|                N|          2|         31|           26|
|   Y|                N|          5|         24|           21|
|   Y|                N|          3|         96|           94|
|   Y|                N|          4|         38|           35|
|   Y|                N|          5|         20|       

In [27]:
# Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
unpaid_df = filtered_votes_df.filter(filtered_votes_df['vine']== 'N')
unpaid_df.show()

+----+-----------------+-----------+-----------+-------------+
|vine|verified_purchase|star_rating|total_votes|helpful_votes|
+----+-----------------+-----------+-----------+-------------+
|   N|                Y|          5|         49|           45|
|   N|                Y|          5|         25|           25|
|   N|                N|          5|         27|           16|
|   N|                Y|          5|         21|           19|
|   N|                Y|          2|         22|           19|
|   N|                Y|          5|         30|           30|
|   N|                Y|          5|         29|           28|
|   N|                Y|          5|         28|           28|
|   N|                Y|          2|         51|           43|
|   N|                Y|          5|         22|           21|
|   N|                Y|          1|         33|           33|
|   N|                Y|          2|         76|           73|
|   N|                Y|          3|         26|       

# **Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).**

In [36]:
# Determine the percentage of 5-star reviews among (paid) Vine reviews
total_paid_vine_reviews = paid_df.count()
total_paid_5star_vine_reviews = paid_df[paid_df['star_rating']== 5].count()
percentage_of_paid_5star_vine_reviews = float(total_paid_5star_vine_reviews) / float(total_paid_vine_reviews)
print(percentage_of_paid_5star_vine_reviews)

0.5909090909090909


In [37]:
# Determine the percentage of 5-star reviews among (unpaid) non-Vine reviews
total_unpaid_nonvine_reviews = unpaid_df.count()
total_unpaid_5star_nonvine_reviews = unpaid_df[unpaid_df['star_rating']== 5].count()
percentage_of_unpaid_5star_nonvine_reviews = float(total_unpaid_5star_nonvine_reviews) / float(total_unpaid_nonvine_reviews)
print(percentage_of_unpaid_5star_nonvine_reviews)

0.5363693630266425


## **Summary for (paid) vine reviews:**

In [43]:
print("Total number of reviews for paid/Vine members:") 
print(total_paid_vine_reviews)
print("")
print("Total number of 5-star reviwes for paid/Vine members:")
print(total_paid_5star_vine_reviews)
print("")
print("Percentage of 5-star reviews for paid/Vine members:")
print(percentage_of_paid_5star_vine_reviews)

Total number of reviews for paid/Vine members:
22

Total number of 5-star reviwes for paid/Vine members:
13

Percentage of 5-star reviews for paid/Vine members:
0.5909090909090909


## **Summary for (unpaid) non-vine reviews:**

In [46]:
print("Total number of reviews for unpaid/non-Vine members:") 
print(total_unpaid_nonvine_reviews)
print("")
print("Total number of 5-star reviwes for unpaid/non-Vine members:")
print(total_unpaid_5star_nonvine_reviews)
print("")
print("Percentage of 5-star reviews for unpaid/non-Vine members:")
print(percentage_of_unpaid_5star_nonvine_reviews)

Total number of reviews for unpaid/non-Vine members:
26987

Total number of 5-star reviwes for unpaid/non-Vine members:
14475

Percentage of 5-star reviews for unpaid/non-Vine members:
0.5363693630266425
